# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Target
target = "Transported"

# --- Feature engineering (similar a labs previos típicos) ---
df = spaceship.copy()

# Cabin -> Deck / CabinNum / Side
df[["Deck", "CabinNum", "Side"]] = df["Cabin"].str.split("/", expand=True)

# PassengerId -> Group (útil a veces)
df["Group"] = df["PassengerId"].str.split("_", expand=True)[0]

# TotalSpend
spend_cols = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
df["TotalSpend"] = df[spend_cols].sum(axis=1)

# Drop columnas con demasiada “identidad”
df = df.drop(columns=["PassengerId", "Name", "Cabin"])

# X / y
X = df.drop(columns=[target])
y = df[target]

# Columnas por tipo
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "bool"]).columns.tolist()

# Preprocess
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

**Perform Train Test Split**

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape

((6954, 15), (1739, 15))

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

base_lr = LogisticRegression(max_iter=2000)

bagging_clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", BaggingClassifier(
        estimator=base_lr,
        n_estimators=200,
        bootstrap=True,      # Bagging
        random_state=42,
        n_jobs=-1
    ))
])

bagging_clf.fit(X_train, y_train)

y_pred_bag = bagging_clf.predict(X_test)
acc_bag = accuracy_score(y_test, y_pred_bag)
acc_bag

0.7901092581943646

- Random Forests

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", RandomForestClassifier(
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    ))
])

rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
acc_rf

0.7952846463484762

- Gradient Boosting

In [10]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", GradientBoostingClassifier(random_state=42))
])

gb_clf.fit(X_train, y_train)

y_pred_gb = gb_clf.predict(X_test)
acc_gb = accuracy_score(y_test, y_pred_gb)
acc_gb

0.7987349051178838

- Adaptive Boosting

In [11]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
        n_estimators=300,
        learning_rate=0.5,
        random_state=42
    ))
])

ada_clf.fit(X_train, y_train)

y_pred_ada = ada_clf.predict(X_test)
acc_ada = accuracy_score(y_test, y_pred_ada)
acc_ada

0.7740080506037953

Which model is the best and why?

In [ ]:
print("Bagging accuracy:", acc_bag)
print("RandomForest accuracy:", acc_rf)
print("GradientBoosting accuracy:", acc_gb)
print("AdaBoost accuracy:", acc_ada)

Bagging accuracy: 0.7901092581943646
RandomForest accuracy: 0.7952846463484762
GradientBoosting accuracy: 0.7987349051178838
AdaBoost accuracy: 0.7740080506037953


🔎 ¿Cuál es el mejor?

Gradient Boosting (≈ 0.799) es el mejor modelo en tu test set.

No por mucho margen, pero es el que más generaliza.

🧠 ¿Por qué gana Gradient Boosting aquí?
Porque: Bagging / Random Forest reducen varianza.
Boosting reduce sesgo.

En datasets como este (Spaceship Titanic), normalmente:
No es un problema de alta varianza extrema.
Es más un problema de capturar patrones complejos entre variables categóricas y numéricas.

Gradient Boosting construye modelos secuencialmente:
Cada árbol corrige los errores del anterior.

Eso le permite:
Ajustar patrones no lineales
Detectar interacciones sutiles
Ajustar mejor fronteras de decisión complejas

🧩 ¿Por qué AdaBoost quedó peor?

AdaBoost es más sensible a:
Ruido
Outliers
Datos mal imputados
Y si el dataset tiene ruido o variables con mucha fragmentación categórica, puede penalizar demasiado ciertos errores.

🎯 Conclusión
Gradient Boosting obtuvo el mejor accuracy en el test set (~0.799).
Este modelo mejora iterativamente corrigiendo los errores de los modelos anteriores, lo que le permite reducir el sesgo y capturar patrones complejos en los datos.
Random Forest también tuvo un desempeño fuerte al reducir varianza mediante bagging, pero Boosting logró una ligera mejora en generalización.